In [52]:
import pandas as pd
from math import log
from surprise import Reader, Dataset
from surprise import KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.dataset import DatasetAutoFolds
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
from itertools import chain
import os

reviews= pd.read_csv("./data/reviews.csv", encoding="cp949")
cafes =  pd.read_csv("./data/cafe_info.csv", encoding="cp949")
# restaurants =  pd.read_csv("./data/restaurant.csv", encoding="cp949")
# drinks =  pd.read_csv("../data/drink.csv", encoding="cp949")

# 1번 유저
my_uid = 1


RESTAURANT_TAGS = [
    '가성비좋은','분위기좋은','친절한','예쁜','고급스러운',
    '푸짐한','데이트','존맛탱','깔끔한','조용한',
    '또먹고싶다','격식있는','동네핫플'
]
CAFE_TAGS = [
    '가성비좋은','분위기좋은','감성카페','고급스러운','조용한',
    '깔끔한','디저트','인테리어','사진찍기좋은','이색적인',
    '뷰가좋은','예쁜','동네핫플'
]
DRINK_TAGS = [
    '가성비좋은','분위기좋은','안주맛집','고급스러운','깔끔한',
    '인테리어','사진찍기좋은','이색적인','동네핫플','취향저격',
    '힙한','감성술집'
]
ACTIVITY_TAGS = [
    '가성비좋은','분위기좋은','재밌는','깔끔한','친절한',
    '인테리어','시간순삭'
]
COURSE_TAGS = [RESTAURANT_TAGS, CAFE_TAGS, DRINK_TAGS, ACTIVITY_TAGS]

#식당과 놀거리 경우, 카테고리 선택하므로 분류해주기
def category_filter(dataFrame, categories):
    data = []
    for idx, row in dataFrame.iterrows():
        for category in categories:
            if category in row['category']:
                data.append(row)
    return pd.DataFrame(data)

#해당 구역만 분류해주기
def dong_filter(dataFrame, dong):
    data = []
    area = [
        ['이태원동'],
        ['한남동'],
        ['용산동'],
        ['한강로동'],
        ['이촌동'],
        ['보광동','동빙고동','서빙고동','주성동'],
        ['원효로동','문배동','신계동','도원동','용문동','신창동','산청동,','청암동'],
        ['서계동','동자동','후암동','갈월동','남영동','청파동','효창동']]
    areaIdx = -1
    for i in range(len(area)):
        if dong in area[i]:
            areaIdx = i
            break

    for idx, row in dataFrame.iterrows():
        if row['addr2'] in area[areaIdx]:
            data.append(row)
    return pd.DataFrame(data)

#식당과 놀거리 경우, 태그선택안하므로 유저리뷰 기준으로 좋아하는 태그를 뽑는다
def tagsFromReviews(userId, course):
    my_reviews = reviews[(reviews['user_id'] == userId) & (reviews['store_id']//1000 == course)]
    dict = {} 
    if len(my_reviews) == 0:
        return COURSE_TAGS[course-1]

    tags = COURSE_TAGS[course-1]
        
    for idx, row in my_reviews.iterrows():
        rating = row['rating']
        if course == 1:
            store = restaurants[restaurants['store_id'] == row['store_id']]
        # activity인 경우
        else:
            store = drinks[drinks['store_id'] == row['store_id']]

        # 해당 store의 가장 많은 리뷰태그 5개
        store_reviews = []
        for tag in tags:
            store_reviews.append([tag,store[tag].values[0]])
        store_reviews.sort(key = lambda x: x[1], reverse=True)
        
        for tag, cnt in store_reviews[:5]:
            if tag not in dict.keys():
                dict[tag] = 0
            if cnt != 0:
                dict[tag] += rating
    
    sorted_dict = sorted(dict.items(), key=lambda x : x[1], reverse=True)
    return [tag[0] for tag in sorted_dict[:3]]

# 식당: 1, 카페: 2, 술집: 3, 활동: 4
def getWhichCourse():
    return 2

# 식당/홛동이면 어느 카테고리 => 태그로 변환해주기
# 카페/술집이면 어느 태그
def getUserSelects():
    # return ['한식','양식']
    return ['분위기좋은', '사진찍기좋은', '깔끔한']

#어느 동을 선택했는지
def getDong():
    return '이태원동'

def tf(t, d):
    # d => 인덱스의 태그들, t => 태그 하나
    return d.count(t)


def idf(spot, t):
    df = 0
    # 전체 태그들중에 해당 태그가 몇번 사용됐는디 로그로
    df += sum(spot[t].values)
    return log(len(spot)/(df+1))

def tfidf(spot, t, d, i):
    # tf는 정의대로 수정
    return spot[t][i] * idf(spot, t)

def compute_score(idx):
    if spots['count'][idx] > 100 : c = 100
    else : c = spots['count'][idx]
    return (c + spots['rating'][idx]*20)/200

def compute_cos_sim(spots, arr):
    cosine_matrix = cosine_similarity(arr, arr)
    spotId = {}
    for idx, c in enumerate(spots['name']):  spotId[idx] = c
    id2spot = {}
    for idx, c in spotId.items() : id2spot[c] = idx
    return cosine_matrix, spotId, id2spot

def get_k_sim(name, k):
    idx = id2spot[name]
    k_spot_by_cbf = {}
    sim_scores = [(i, c + compute_score(i) ) for i, c in enumerate(cosine_matrix[idx]) if i != idx]
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)
    for i, score in sim_scores[0:k]:
        k_spot_by_cbf[spotId[i]] = score * 2.5
    return k_spot_by_cbf

def get_top_n(predictions, n=10):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def make_uid_iid_list(uid, dongId):
    uid_iid_list = []
    for idx, r in cafes.iterrows():
        if r['addr2'] == dongId:
            uid_iid_list.append([my_uid, r['store_id'], None])
    return uid_iid_list

def get_k_spot_by_cf(k):
    top_n = get_top_n(pred, n=k)
    k_spot_by_cf = {}
    for uid, user_ratings in top_n.items():
        for (iid, est) in user_ratings:
            k_spot_by_cf[cafes[cafes['store_id']==iid]['name'].item()] = est                
    return k_spot_by_cf

# 1. 동 선택
dong = getDong()

# 2. 코스 선택 (식당/카페/술집/활동)
course = getWhichCourse()
courseTags = COURSE_TAGS[course-1]

# 3. 카테고리 / 태그 선택
categories = getUserSelects() # 어느 카테고리/태그를 선택했는가
selectedTags = []
if course == 1 or course == 4:
    selectedTags = tagsFromReviews(2, course)
else:
    selectedTags = categories

# 4. 선택된 코스에 맞는 데이터 불러오기
spots = cafes

# 5. spot 분류
if course == 1 or course == 4: # 식당/활동의 경우 카테고리로 분류
    spots = category_filter(restaurants, categories) 
spots = dong_filter(spots, dong) # 어느동을 선택했는지
spots.reset_index(inplace=True, drop=True)

# 6. TF-IDF 구하기
result = []
for i in spots.index:
    result.append([])
    d = spots['name'][i]
    for j in range(len(courseTags)):
        t = courseTags[j]
        result[-1].append(tfidf(spots, t, d, i))

tfidf_ = pd.DataFrame(result, columns=courseTags)

# 선택된 태그는 1, 아니면 0으로 TF-IDF data frame에 넣어주기
N = len(spots)
temp = []
for tag in courseTags:
    if tag in selectedTags:
        temp.append(1)
    else:
        temp.append(0)

tfidf_.loc[N] = temp
if course == 1 or course == 4: # 식당 / 활동
    temp = ['','temp','','','','','','','','',5,1] + temp + [''] 
else: # 카페 / 술집
    temp = ['','','temp','','','','','','','',5,1] + temp + ['']
spots.loc[N] = temp
N += 1

# 코사인 유사도 구하기
cosine_matrix, spotId, id2spot = compute_cos_sim(spots, tfidf_)

cbf_return = get_k_sim("temp",1000)



# CF
reviews = reviews[['user_id', 'store_id', 'rating']]
cafes = cafes[['store_id','category_id','name','addr2','rating','count','가성비좋은','분위기좋은','감성카페','고급스러운','조용한',
    '깔끔한','디저트','인테리어','사진찍기좋은','이색적인','뷰가좋은','예쁜','동네핫플']]

reader = Reader(rating_scale=(1, 5.0))
data = Dataset.load_from_df(reviews[['user_id', 'store_id', 'rating']], reader)

trainset = data.build_full_trainset()

option = {'name':'pearson', 'user_based':True}
algo = KNNBasic(sim_options=option)

algo.fit(trainset)

uid_iid_list = make_uid_iid_list(my_iid, dong)
pred = algo.test(uid_iid_list)
cf_return = get_k_spot_by_cf(1000)

rec_for_user = defaultdict(list)
for k, v in chain(cbf_return.items(), cf_return.items()):
    rec_for_user[k].append(v)

for k, v in rec_for_user.items():
    rec_for_user[k] = sum(v)
    
rec_for_user = sorted(rec_for_user.items(), key=lambda x : x[1], reverse=True)
for recspot in rec_for_user[:20]:
    print(recspot[0])

Computing the pearson similarity matrix...
Done computing similarity matrix.
더미쓰리
바움
리젠트스카이 루프탑 카페
녹사커피
위긴티식스
더베이커스테이블
투썸플레이스 이태원역점
The crumpet house
스노잉 이태원경리단길점
카페 대추와 자몽
배스킨라빈스 경리단길점
헤미안
커피나인 이태원
무진장
카페무제
애브리씽벗더히어로카페
안티 스트레스
아이도넛케어
스타벅스 이태원역점
푸실케이크


In [49]:
cbf_return

{'바움': 2.8139273332583645,
 '리젠트스카이 루프탑 카페': 2.8105433259915835,
 '녹사커피': 2.7447216606025067,
 '위긴티식스': 2.7247043091710315,
 '더베이커스테이블': 2.7154871272452654,
 '투썸플레이스 이태원역점': 2.640509207149871,
 'The crumpet house': 2.5224306622744743,
 '스노잉 이태원경리단길점': 2.510569749403496,
 '카페 대추와 자몽': 2.4792546799001816,
 '배스킨라빈스 경리단길점': 2.4594668729737155,
 '헤미안': 2.4530282980175593,
 '커피나인 이태원': 2.3603985327390555,
 '무진장': 2.3517483169651,
 '카페무제': 2.331947411052756,
 '애브리씽벗더히어로카페': 2.312624776737019,
 '안티 스트레스': 2.304043231704057,
 '아이도넛케어': 2.265836934846479,
 '스타벅스 이태원역점': 2.259730035833421,
 '더미쓰리': 2.2545506596168567,
 '푸실케이크': 2.2448761194485827,
 'TOKKi': 2.2163950522470355,
 '플랜티카': 2.1987739789610172,
 '가든스튜디오': 2.195273466902245,
 '55브랜드 store&cafe': 2.1809056007300445,
 '코지빌라커피': 2.1723192376299605,
 '브라이틀링 카페': 2.171599679167029,
 '서울루덴스': 2.170087295980278,
 '산수목모모스': 2.1683203951701113,
 '와플대학 이태원캠퍼스': 2.1655489852016614,
 '파치드 서울': 2.1637550001234094,
 '플랜트': 2.1484270617721988,
 '님부스커피

In [50]:
cf_return

{'더미쓰리': 5.0,
 '1b coffee stand': 4.275,
 '55브랜드 store&cafe': 4.275,
 'Apt Seoul': 4.275,
 'Cafe Flat': 4.275,
 'Cafe TRVR': 4.275,
 'EverythingButTheHero': 4.275,
 'Kosher Eli Food': 4.275,
 'TOKKi': 4.275,
 'The crumpet house': 4.275,
 '가든스튜디오': 4.275,
 '그레트힐란': 4.275,
 '꾹': 4.275,
 '네버더레스': 4.275,
 '네이버스': 4.275,
 '녹사다리': 4.275,
 '녹사커피': 4.275,
 '님부스커피': 4.275,
 '다미': 4.275,
 '다바': 4.275,
 '더베이커스테이블': 4.275,
 '더코지코너': 4.275,
 '더팔러서울': 4.275,
 '델리카': 4.275,
 '도코로': 4.275,
 '뚜레쥬르 이태원점': 4.275,
 '뚜레쥬르 카페 이태원역점': 4.275,
 '라비앙코코': 4.275,
 '레잇먼트': 4.275,
 '로스니카페': 4.275,
 '로컬빌라베이글': 4.275,
 '리젠트스카이 루프탑 카페': 4.275,
 '리지트커피': 4.275,
 '마린커피': 4.275,
 '매머드익스프레스 이태원동점': 4.275,
 '먼셀커피 이태원점': 4.275,
 '무진장': 4.275,
 '미니마이즈': 4.275,
 '바움': 4.275,
 '반지더하기 이태원점': 4.275,
 '발루토피비': 4.275,
 '배스킨라빈스 경리단길점': 4.275,
 '버뮤다 드링크트럭': 4.275,
 '버클리커피소셜': 4.275,
 '벤스쿠키 이태원점': 4.275,
 '벨라스 홈스테이': 4.275,
 '보마켓 경리단점': 4.275,
 '브라이틀링 카페': 4.275,
 '블랙마켓': 4.275,
 '블루스': 4.275,
 '빵어니스타 이태원점': 4.275,
 '빽다방 이태원점': 4.275